In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Download a Folder**

In [25]:
import shutil
from google.colab import files

# Specify the folder path to download
folder_path_to_download = '/content/features_dataset_vgg16'

# Create a zip file
shutil.make_archive('downloaded_folder', 'zip', folder_path_to_download)

# Move the zip file to /content
shutil.move('downloaded_folder.zip', '/content/downloaded_folder.zip')

# Provide a download link for the zip file
files.download('/content/downloaded_folder.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Delete folder:**

In [ ]:
# import shutil
# import os

# def delete_folder(folder_path):
#     try:
#         shutil.rmtree(folder_path)
#         print(f"Folder '{folder_path}' and its contents have been successfully deleted.")
#     except FileNotFoundError:
#         print(f"Folder '{folder_path}' not found.")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Example usage: Delete a folder named 'example_folder'
# folder_to_delete = '/content/features_dataset'  # Replace with the actual path of the folder you want to delete
# delete_folder(folder_to_delete)

Folder '/content/features_dataset1' and its contents have been successfully deleted.


# **preprocess dataset**

In [2]:
!pip install dlib scikit-image

In [3]:
import zipfile
import os
import cv2
import dlib
from skimage import exposure
from skimage import img_as_ubyte  # Added import for img_as_ubyte

# Specify the name of your ZIP file
zip_file_name = "/content/drive/MyDrive/Project/Classmates.zip"

# Specify the directory where you want to extract the contents
extracted_folder = "/content/Classmates"

# Create the target directory if it doesn't exist
os.makedirs(extracted_folder, exist_ok=True)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Check the extracted contents
extracted_contents = os.listdir(extracted_folder)
print("Extracted contents:", extracted_contents)

# Load face detector from dlib
face_detector = dlib.get_frontal_face_detector()

# Path to the directory containing raw images
raw_dataset_path = "/content/Classmates"

# Path to the directory where preprocessed images will be saved
preprocessed_dataset_path = "/content/preprocessed_dataset"
os.makedirs(preprocessed_dataset_path, exist_ok=True)

# Iterate over class folders
for class_folder in os.listdir(raw_dataset_path):
    class_path = os.path.join(raw_dataset_path, class_folder)

    # Create a folder for each class in the preprocessed dataset
    preprocessed_class_path = os.path.join(preprocessed_dataset_path, class_folder)
    os.makedirs(preprocessed_class_path, exist_ok=True)

    # Iterate over images in the class folder
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)

        # Read the image
        img = cv2.imread(image_path)

        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Detect faces using dlib
        faces = face_detector(gray)

        # Process each detected face
        for i, face in enumerate(faces):
            x, y, w, h = face.left(), face.top(), face.width(), face.height()

            # Crop the face
            cropped_face = img[y:y+h, x:x+w]

            # Resize the face to a standard size
            resized_face = cv2.resize(cropped_face, (224, 224))

            # Normalize pixel values to a standard range (0 to 1)
            normalized_face = resized_face / 255.0

            # Save the preprocessed face
            preprocessed_image_name = f"{class_folder}_face{i+1}_{image_name}"
            preprocessed_image_path = os.path.join(preprocessed_class_path, preprocessed_image_name)

            # Save the normalized face without converting to ubyte
            cv2.imwrite(preprocessed_image_path, normalized_face)

            # Augmentation: You can add additional data augmentation steps here if needed

print("Preprocessing complete.")


Extracted contents: ['Umer', 'Shahab', 'Annas', 'Tehseen', 'Taha', 'Adan', 'Shah Wali']
Preprocessing complete.


# **Feature Extraction**

## **Feature Extraction Using VGG16**

In [4]:
!pip install tensorflow numpy

In [5]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
import os
import numpy as np

# Path to the directory where preprocessed images are stored
preprocessed_dataset_path = "/content/preprocessed_dataset"

# Path to the directory where you want to save the extracted features
features_dataset_path = "/content/features_dataset_vgg16"
os.makedirs(features_dataset_path, exist_ok=True)

# Load the VGG16 model pre-trained on 'imagenet' dataset
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features using the VGG16 model
def extract_features(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = base_model.predict(img_array)
    return features.flatten()

# Iterate over preprocessed images and extract features
for class_folder in os.listdir(preprocessed_dataset_path):
    class_path = os.path.join(preprocessed_dataset_path, class_folder)

    # Create a folder for each class in the features dataset
    features_class_path = os.path.join(features_dataset_path, class_folder)
    os.makedirs(features_class_path, exist_ok=True)

    # Iterate over preprocessed images in the class folder
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)

        # Extract features using VGG16
        features = extract_features(image_path)

        # Save the extracted features
        features_file_name = f"{class_folder}_features_{image_name.replace('.jpg', '.npy')}"
        features_file_path = os.path.join(features_class_path, features_file_name)
        np.save(features_file_path, features)

print("Feature extraction complete.")

1/1 [==============================] - 1s 545ms/step
Feature extraction complete.


## **Feature extraction using FaceNet**

In [6]:
!pip install keras_vggface
!pip install tensorflow numpy keras-vggface
!pip install keras_applications
!pip install facenet-pytorch
!pip install mtcnn
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 721.1 kB/s eta 0:00:00


In [8]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import InceptionResnetV1, MTCNN, extract_face

# Load pre-trained face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval()

# Path to the directory where preprocessed images are stored
preprocessed_dataset_path = "/content/preprocessed_dataset"

# Path to the directory where you want to save the extracted features
features_dataset_path = "/content/features_dataset_facenet"
os.makedirs(features_dataset_path, exist_ok=True)

# Function to extract features using facenet
def extract_features_facenet(img_path):
    img = cv2.imread(img_path)

    # Convert BGR image to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resize the face to a standard size
    resized_face = cv2.resize(img_rgb, (160, 160))

    # Normalize pixel values to a standard range (0 to 1)
    normalized_face = resized_face / 255.0

    # Convert to PyTorch tensor
    tensor_face = torch.from_numpy(normalized_face).unsqueeze(0).permute(0, 3, 1, 2).float()

    # Forward pass to get the features
    features = model(tensor_face)

    return features.squeeze().detach().numpy()

# Update the feature extraction loop
for class_folder in os.listdir(preprocessed_dataset_path):
    class_path = os.path.join(preprocessed_dataset_path, class_folder)

    # Create a folder for each class in the features dataset
    features_class_path = os.path.join(features_dataset_path, class_folder)
    os.makedirs(features_class_path, exist_ok=True)

    # Iterate over preprocessed images in the class folder
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)

        # Extract features using facenet
        features = extract_features_facenet(image_path)

        print(f"Processing {image_name}")
        print(f"Shape of features extracted: {features.shape}")

        # Save the extracted features
        features_file_name = f"{class_folder}_features_{image_name.replace('.jpg', '.npy')}"
        features_file_path = os.path.join(features_class_path, features_file_name)
        np.save(features_file_path, features)

print("Facenet Feature extraction complete.")


  0%|          | 0.00/107M [00:00<?, ?B/s]

Processing Umer_face1_WhatsApp Image 2024-01-02 at 6.06.38 PM.jpeg
Shape of features extracted: (512,)
Processing Umer_face1_WhatsApp Image 2024-01-02 at 6.06.48 PM (1).jpeg
Shape of features extracted: (512,)
Processing Umer_face1_WhatsApp Image 2024-01-02 at 6.06.47 PM.jpeg
Shape of features extracted: (512,)
Processing Umer_face1_WhatsApp Image 2024-01-02 at 6.06.48 PM.jpeg
Shape of features extracted: (512,)
Processing Umer_face1_WhatsApp Image 2024-01-02 at 6.06.41 PM.jpeg
Shape of features extracted: (512,)
Processing Shahab_face1_WhatsApp Image 2024-01-05 at 8.10.48 PM.jpeg
Shape of features extracted: (512,)
Processing Shahab_face1_WhatsApp Image 2024-01-02 at 6.07.07 PM (1).jpeg
Shape of features extracted: (512,)
Processing Shahab_face1_WhatsApp Image 2024-01-02 at 6.07.08 PM.jpeg
Shape of features extracted: (512,)
Processing Shahab_face1_WhatsApp Image 2024-01-02 at 6.07.05 PM.jpeg
Shape of features extracted: (512,)
Processing Shahab_face1_WhatsApp Image 2024-01-05 at 7.30

# **Train a Classifier on VGG16 extracted features data**

## **Train SVM**

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# Load saved features and corresponding labels
X, y = [], []
features_dataset_path = '/content/features_dataset_vgg16'
for class_folder in os.listdir(features_dataset_path):
    class_path = os.path.join(features_dataset_path, class_folder)
    for features_file_name in os.listdir(class_path):
        features_file_path = os.path.join(class_path, features_file_name)
        features = np.load(features_file_path)
        X.append(features)
        y.append(class_folder)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVM classifier
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Print the classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

# Save the trained SVM model
model_filename = "svm_model.pkl"

# Check if a saved model exists, delete it if it does
if os.path.exists(model_filename):
    os.remove(model_filename)

# Save the new model
joblib.dump(classifier, model_filename)
print(f"Model saved as {model_filename}")


Accuracy: 0.25
Confusion Matrix:
[[1 0 1 0 0 2 0]
 [0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0]
 [0 1 0 0 0 2 0]
 [1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]]
Classification Report:
              precision    recall  f1-score   support

        Adan       0.33      0.25      0.29         4
       Annas       0.00      0.00      0.00         0
   Shah Wali       0.50      0.50      0.50         2
      Shahab       0.00      0.00      0.00         3
        Taha       1.00      0.50      0.67         2
     Tehseen       0.00      0.00      0.00         0
        Umer       0.00      0.00      0.00         1

    accuracy                           0.25        12
   macro avg       0.26      0.18      0.21        12
weighted avg       0.36      0.25      0.29        12

Model saved as svm_model.pkl


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

## **Train RandomForest**

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# Load saved features and corresponding labels
X, y = [], []
features_dataset_path = '/content/features_dataset_vgg16'
for class_folder in os.listdir(features_dataset_path):
    class_path = os.path.join(features_dataset_path, class_folder)
    for features_file_name in os.listdir(class_path):
        features_file_path = os.path.join(class_path, features_file_name)
        features = np.load(features_file_path)
        X.append(features)
        y.append(class_folder)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest classifier
classifier_rf = RandomForestClassifier(n_estimators=100, random_state=42)
classifier_rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = classifier_rf.predict(X_test)

# Evaluate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Classifier Accuracy: {accuracy_rf}")

# Print the confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:")
print(conf_matrix_rf)

# Print the classification report
class_report_rf = classification_report(y_test, y_pred_rf)
print("Classification Report:")
print(class_report_rf)

# Save the trained RandomForest model
model_filename_rf = "randomforest_model.pkl"

# Check if a saved model exists, delete it if it does
if os.path.exists(model_filename_rf):
    os.remove(model_filename_rf)

# Save the new model
joblib.dump(classifier_rf, model_filename_rf)
print(f"RandomForest Model saved as {model_filename_rf}")

Random Forest Classifier Accuracy: 0.3333333333333333
Confusion Matrix:
[[1 0 0 0 3 0]
 [1 0 0 0 1 0]
 [0 0 3 0 0 0]
 [2 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 0]]
Classification Report:
              precision    recall  f1-score   support

        Adan       0.25      0.25      0.25         4
   Shah Wali       0.00      0.00      0.00         2
      Shahab       1.00      1.00      1.00         3
        Taha       0.00      0.00      0.00         2
     Tehseen       0.00      0.00      0.00         0
        Umer       0.00      0.00      0.00         1

    accuracy                           0.33        12
   macro avg       0.21      0.21      0.21        12
weighted avg       0.33      0.33      0.33        12

RandomForest Model saved as randomforest_model.pkl


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

# **Train a Classifier on Facenet extracted features data**

## **Train SVM**

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# Load saved features and corresponding labels
X, y = [], []
features_dataset_path = '/content/features_dataset_facenet'
for class_folder in os.listdir(features_dataset_path):
    class_path = os.path.join(features_dataset_path, class_folder)
    for features_file_name in os.listdir(class_path):
        features_file_path = os.path.join(class_path, features_file_name)
        features = np.load(features_file_path)
        X.append(features)
        y.append(class_folder)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVM classifier
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Print the classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

# Save the trained SVM model
model_filename = "svm_model.pkl"

# Check if a saved model exists, delete it if it does
if os.path.exists(model_filename):
    os.remove(model_filename)

# Save the new model
joblib.dump(classifier, model_filename)
print(f"Model saved as {model_filename}")

Accuracy: 0.0
Confusion Matrix:
[[0 0 0 0 4 0]
 [0 0 0 0 2 0]
 [0 0 0 0 3 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 0]]
Classification Report:
              precision    recall  f1-score   support

        Adan       0.00      0.00      0.00       4.0
   Shah Wali       0.00      0.00      0.00       2.0
      Shahab       0.00      0.00      0.00       3.0
        Taha       0.00      0.00      0.00       2.0
     Tehseen       0.00      0.00      0.00       0.0
        Umer       0.00      0.00      0.00       1.0

    accuracy                           0.00      12.0
   macro avg       0.00      0.00      0.00      12.0
weighted avg       0.00      0.00      0.00      12.0

Model saved as svm_model.pkl


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

## **Train RandomForest**


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# Load saved features and corresponding labels
X, y = [], []
features_dataset_path = '/content/features_dataset_facenet'
for class_folder in os.listdir(features_dataset_path):
    class_path = os.path.join(features_dataset_path, class_folder)
    for features_file_name in os.listdir(class_path):
        features_file_path = os.path.join(class_path, features_file_name)
        features = np.load(features_file_path)
        X.append(features)
        y.append(class_folder)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest classifier
classifier_rf = RandomForestClassifier(n_estimators=100, random_state=42)
classifier_rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = classifier_rf.predict(X_test)

# Evaluate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Classifier Accuracy: {accuracy_rf}")

# Print the confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:")
print(conf_matrix_rf)

# Print the classification report
class_report_rf = classification_report(y_test, y_pred_rf)
print("Classification Report:")
print(class_report_rf)

# Save the trained RandomForest model
model_filename_rf = "randomforest_model.pkl"

# Check if a saved model exists, delete it if it does
if os.path.exists(model_filename_rf):
    os.remove(model_filename_rf)

# Save the new model
joblib.dump(classifier_rf, model_filename_rf)
print(f"RandomForest Model saved as {model_filename_rf}")

Random Forest Classifier Accuracy: 0.08333333333333333
Confusion Matrix:
[[0 0 0 0 0 4 0]
 [0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0]
 [1 0 0 1 0 0 1]
 [0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]]
Classification Report:
              precision    recall  f1-score   support

        Adan       0.00      0.00      0.00         4
       Annas       0.00      0.00      0.00         0
   Shah Wali       0.00      0.00      0.00         2
      Shahab       1.00      0.33      0.50         3
        Taha       0.00      0.00      0.00         2
     Tehseen       0.00      0.00      0.00         0
        Umer       0.00      0.00      0.00         1

    accuracy                           0.08        12
   macro avg       0.14      0.05      0.07        12
weighted avg       0.25      0.08      0.12        12



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

RandomForest Model saved as randomforest_model.pkl
